In [11]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
# import argparse
# parser=argparse.ArgumentParser()
# parser.add_argument("--task", default="Return a list of numbers")
# parser.add_argument("--language", default="python")
# args=parser.parse_args()

# below in this part of code: replace with args.parameters
#     result=code_chain.invoke({
#     "language":args.language,
#     "task":args.task
# })
# result

# I can use this in a .py file to run - python app.py --language javascript --task 'print hello'

In [20]:
llm=ChatGroq(model='llama-3.1-70b-versatile')
code_prompt=PromptTemplate(
    template="Write a very short {language} function that will {task}",
    input_variables=["language","task"]
)

code_chain=LLMChain(
    llm=llm,
    prompt=code_prompt
)

In [22]:
result=code_chain.invoke({
    "language":"python",
    "task":"return a list of numbers"
})
result

{'language': 'python',
 'task': 'return a list of numbers',
 'text': '### Function to Generate a List of Numbers\n\n```python\ndef generate_numbers(start, end):\n    """\n    Generate a list of numbers from start to end (inclusive).\n\n    Args:\n        start (int): The starting number of the list.\n        end (int): The ending number of the list.\n\n    Returns:\n        list: A list of numbers from start to end.\n    """\n    return list(range(start, end + 1))\n\n# Example usage:\nnumbers = generate_numbers(1, 10)\nprint(numbers)  # Output: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n```\n\nThis function uses the built-in `range` function to generate a sequence of numbers from `start` to `end` (inclusive), and then converts it to a list.'}

In [23]:
print(result['text'])

### Function to Generate a List of Numbers

```python
def generate_numbers(start, end):
    """
    Generate a list of numbers from start to end (inclusive).

    Args:
        start (int): The starting number of the list.
        end (int): The ending number of the list.

    Returns:
        list: A list of numbers from start to end.
    """
    return list(range(start, end + 1))

# Example usage:
numbers = generate_numbers(1, 10)
print(numbers)  # Output: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
```

This function uses the built-in `range` function to generate a sequence of numbers from `start` to `end` (inclusive), and then converts it to a list.


In [24]:
code_chain=LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code"
)

result=code_chain.invoke({
    "language":"python",
    "task":"return a list of numbers"
})

result['code'] # instead of result['text]

'### List of Numbers Function\n\nHere is a simple Python function that returns a list of numbers from 1 to `n`.\n\n```python\ndef get_numbers(n):\n    """\n    Returns a list of numbers from 1 to n.\n\n    Args:\n        n (int): The maximum number in the list.\n\n    Returns:\n        list: A list of numbers from 1 to n.\n    """\n    return list(range(1, n + 1))\n```\n\n### Example Use Case\n\n```python\n# Get a list of numbers from 1 to 10\nnumbers = get_numbers(10)\nprint(numbers)  # Output: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n```'

# Sequential chain

In [26]:
#second prompt in addition to previous prompt
test_prompt=PromptTemplate(
    input_variables=["language","code"],
    template="Write a test for the following {language} code:\n{code}"
)

test_chain=LLMChain(llm=llm, prompt=test_prompt, output_key="test")

In [27]:
from langchain.chains import SequentialChain

chain=SequentialChain(
    chains=[code_chain, test_chain],
    input_variables=["task","language"],
    output_variables=["test","code"]
)

result=chain({
    "language":"python",
    "task":"Write a program to add two numbers"
})

C:\Users\varsh\AppData\Local\Temp\ipykernel_8848\4273785881.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain({


In [28]:
result

{'language': 'python',
 'task': 'Write a program to add two numbers',
 'test': '### Testing the `add_two_numbers` Function\n\nTo ensure the `add_two_numbers` function works correctly, we\'ll write test cases using Python\'s built-in `unittest` module. We\'ll cover different scenarios, including:\n\n* Positive numbers\n* Negative numbers\n* Zero\n* Edge cases (large numbers)\n\n```python\nimport unittest\n\nclass TestAddTwoNumbers(unittest.TestCase):\n    def test_positive_numbers(self):\n        self.assertEqual(add_two_numbers(5, 7), 12)\n        self.assertEqual(add_two_numbers(10, 3), 13)\n\n    def test_negative_numbers(self):\n        self.assertEqual(add_two_numbers(-5, -7), -12)\n        self.assertEqual(add_two_numbers(-10, -3), -13)\n\n    def test_mixed_numbers(self):\n        self.assertEqual(add_two_numbers(-5, 7), 2)\n        self.assertEqual(add_two_numbers(10, -3), 7)\n\n    def test_zero(self):\n        self.assertEqual(add_two_numbers(0, 0), 0)\n        self.assertEqua

In [29]:
print(result['code'])

### Adding Two Numbers in Python
#### Function

```python
def add_two_numbers(num1, num2):
    """
    Returns the sum of two numbers.

    Args:
        num1 (int): The first number.
        num2 (int): The second number.

    Returns:
        int: The sum of num1 and num2.
    """
    return num1 + num2
```

#### Example Use Case

```python
# Call the function with two numbers
result = add_two_numbers(5, 7)
print("The sum is:", result)  # Output: The sum is: 12
```

This function takes two integers as input and returns their sum. It's a simple example of how to encapsulate a calculation in a reusable function.


In [30]:
print(result['test'])

### Testing the `add_two_numbers` Function

To ensure the `add_two_numbers` function works correctly, we'll write test cases using Python's built-in `unittest` module. We'll cover different scenarios, including:

* Positive numbers
* Negative numbers
* Zero
* Edge cases (large numbers)

```python
import unittest

class TestAddTwoNumbers(unittest.TestCase):
    def test_positive_numbers(self):
        self.assertEqual(add_two_numbers(5, 7), 12)
        self.assertEqual(add_two_numbers(10, 3), 13)

    def test_negative_numbers(self):
        self.assertEqual(add_two_numbers(-5, -7), -12)
        self.assertEqual(add_two_numbers(-10, -3), -13)

    def test_mixed_numbers(self):
        self.assertEqual(add_two_numbers(-5, 7), 2)
        self.assertEqual(add_two_numbers(10, -3), 7)

    def test_zero(self):
        self.assertEqual(add_two_numbers(0, 0), 0)
        self.assertEqual(add_two_numbers(5, 0), 5)
        self.assertEqual(add_two_numbers(0, 7), 7)

    def test_large_numbers(sel